In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import category_encoders as ce
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_log_error
import optuna
import optuna.visualization as vis
from optuna.integration import XGBoostPruningCallback
pd.set_option('future.no_silent_downcasting', True)

In [ ]:
# Обработка и объединение train данных
train = train_df.merge(stores_df, on='store_nbr', how='left')
train = train.merge(transactions_df, on=['date', 'store_nbr'], how='left')
train = train.merge(oil_df, on='date', how='left')
train = train.merge(holidays_df, on='date', how='left')

train['oil_price'] = train['dcoilwtico'].ffill().bfill()
train['holiday_type'] = train['type_y'].fillna('No Holiday')
train['transactions'] = train['transactions'].ffill()
train.drop(columns=['type_y'], inplace=True)

train['store_nbr'] = train['store_nbr'].astype(str)
train['cluster'] = train['cluster'].astype(str)

# Создание временных признаков
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day_of_week'] = train['date'].dt.dayofweek
train['week_of_year'] = train['date'].dt.isocalendar().week
train['is_weekend'] = train['day_of_week'].isin([5, 6]).astype(int)
train['is_holiday'] = (train['holiday_type'] != 'Work Day').astype(int)

# Лаговые и скользящие средние признаки
train['lag_7_sales'] = train['sales'].shift(7)
train['lag_14_sales'] = train['sales'].shift(14)
train['rolling_mean_7'] = train['sales'].shift(1).rolling(window=7).mean()
train['rolling_mean_14'] = train['sales'].shift(1).rolling(window=14).mean()

# Удаление строк с пропущенными значениями
train = train.dropna(subset=['lag_7_sales', 'lag_14_sales', 'rolling_mean_7', 'rolling_mean_14'])

# Лог-трансформация целевой переменной
train['log_sales'] = np.log1p(train['sales'])

# Обработка и объединение test данных
test = test_df.merge(stores_df, on='store_nbr', how='left')
test = test.merge(transactions_df, on=['date', 'store_nbr'], how='left')
test = test.merge(oil_df, on='date', how='left')
test = test.merge(holidays_df, on='date', how='left')

test['oil_price'] = test['dcoilwtico'].ffill().bfill()
test['holiday_type'] = test['type_y'].fillna('No Holiday')
test['transactions'] = test['transactions'].ffill()
test.drop(columns=['type_y'], inplace=True)

test['store_nbr'] = test['store_nbr'].astype(str)
test['cluster'] = test['cluster'].astype(str)

# Создание временных признаков в test
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day_of_week'] = test['date'].dt.dayofweek
test['week_of_year'] = test['date'].dt.isocalendar().week
test['is_weekend'] = test['day_of_week'].isin([5, 6]).astype(int)
test['is_holiday'] = (test['holiday_type'] != 'Work Day').astype(int)

# Лаговые и скользящие средние признаки в test
test['lag_7_sales'] = train['sales'].shift(7)  # Нет данных продаж для test, используем train
test['lag_14_sales'] = train['sales'].shift(14)
test['rolling_mean_7'] = train['sales'].shift(1).rolling(window=7).mean()
test['rolling_mean_14'] = train['sales'].shift(1).rolling(window=14).mean()

# Удаление строк с пропущенными значениями
test = test.dropna(subset=['lag_7_sales', 'lag_14_sales', 'rolling_mean_7', 'rolling_mean_14'])



In [ ]:
# Разделение train на признаки и целевую переменную
X = train.drop(columns=['sales', 'log_sales'])
y = train['log_sales']

# Target Encoding для категориальных признаков
categorical_features = ['holiday_type', 'locale', 'locale_name', 'store_nbr', 'family', 'city', 'state', 'cluster']
numerical_features = ['onpromotion', 'transactions', 'oil_price', 'lag_7_sales', 'lag_14_sales', 'rolling_mean_7', 'rolling_mean_14']

target_encoder = ce.TargetEncoder(cols=categorical_features)

# Трансформация числовых признаков
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Препроцессор для всех признаков
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', target_encoder, categorical_features)
    ]
)

In [ ]:
# Функция для Optuna
def objective(trial):
    
    param = {
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
        'objective': 'reg:squarederror',
        'tree_method': 'hist',
        'device': 'cuda',
        'random_state': 42
    }

    tscv = TimeSeriesSplit(n_splits=7)
    rmsle_scores = []

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        X_train_encoded = target_encoder.fit_transform(X_train, y_train)
        X_test_encoded = target_encoder.transform(X_test)

        # Выполнение предобработки
        X_train_processed = preprocessor.fit_transform(X_train_encoded, y_train)
        X_test_processed = preprocessor.transform(X_test_encoded)

        dtrain = xgb.DMatrix(X_train_processed, label=y_train)
        dtest = xgb.DMatrix(X_test_processed, label=y_test)

        # Обучение модели с ранней остановкой
        model = xgb.train(
            param, 
            dtrain,
            num_boost_round=trial.suggest_int('n_estimators', 100, 1000),
            evals=[(dtest, 'validation')],
            early_stopping_rounds=15,
            verbose_eval=False,
        )

        # Предсказание
        y_pred = model.predict(dtest)
        y_pred = np.maximum(0, y_pred)

        # Применение RMSLE
        rmsle = mean_squared_log_error(y_test, y_pred, squared=False)
        rmsle_scores.append(rmsle)
        
    return sum(rmsle_scores) / len(rmsle_scores)

# Запуск Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=40)

print("Best trial:")
trial = study.best_trial
print(f"  RMSLE: {trial.value}")
print("  Best hyperparameters: ", trial.params)


In [ ]:
# Финальное предсказание на тестовом наборе
X_test = test.drop(columns=['id'])

X_test_encoded = target_encoder.transform(X_test)
X_test_processed = preprocessor.transform(X_test_encoded)

dtest = xgb.DMatrix(X_test_processed)
y_pred = model.predict(dtest)
y_pred = np.maximum(0, y_pred)
